This will be a script that downloads ancillary data files from AWS, in order to write the sampling procedure

In [143]:
import boto3
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import requests
import netCDF4
import geopandas as gpd
import numpy as np

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))


In [190]:
landsat_bucket = 'usgs-landsat'

landsat_contents = s3_client.list_objects(Bucket=landsat_bucket, Prefix = 'collection02/')['Contents']

ClientError: An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

# Copernicus Data:

Writing script to pull all grid cells in CONUS

In [172]:
dem_bucket = 'copernicus-dem-30m'

dem_contents = s3_client.list_objects(Bucket=dem_bucket, Prefix = '2019')['Contents']


In [185]:
s3_resource = boto3.resource('s3')

bucket = s3_resource.Bucket('copernicus-dem-30m')

bucket.objects.all()

s3.Bucket.objectsCollection(s3.Bucket(name='copernicus-dem-30m'), s3.ObjectSummary)

In [249]:
import awswrangler as wr

dem_keys = wr.s3.list_objects('s3://' + dem_bucket)


In [ ]:
from itertools import compress
def is_n_w(key):
    split = key.split('_')
    if len(split) >= 4:
        if split[4].startswith('N') and split[6].startswith('W'):
            return True
        return False
    else:
        return key
    
    
dem_keys_pure = [obj for obj in dem_keys if len(obj.split('_')) >= 4]

filter_list = [is_n_w(k) for k in dem_keys_pure]

dem_keys_pure = list(compress(dem_keys_pure,filter_list))

In [345]:
degrees = []
key_list = []
for n in range(23,50):
    for w in range(65,125):
        degrees.append((str(n),str(w).zfill(3)))
        

for n,w in degrees:
    rand_key = objects[0].split("_")
    
    rand_key[4] = 'N' + n
    rand_key[6] = 'W' + w
    
    rand_key[12] = 'N' + n
    rand_key[14] = 'W' + w
    
    key_list.append('_'.join(rand_key))
    
key_list = ['s3://copernicus-dem-30m/' + k for k in key_list]

In [353]:
master_key_list = [k for k in dem_keys_pure if k in key_list]

In [285]:
def get_n_w(key):
    split = key.split("_")
    if len(split) >= 4:
        return (split[4], split[6])
    else:
        return key

In [354]:
norths_wests = [get_n_w(k) for k in dem_keys]
meta_data = [obj for obj in norths_wests if type(obj) != tuple]
norths_wests = [obj for obj in norths_wests if type(obj) == tuple]

wests_only = [obj for obj in norths_wests if obj[1].startswith('W')]

degrees_formatted = [('N'+n, 'W'+w) for (n,w) in degrees]

In [356]:
# Now to find the URLs

In [230]:
f'https://{dem_bucket}.s3.amazonaws.com/{key_list[0]}'

'https://copernicus-dem-30m.s3.amazonaws.com/2019/Copernicus_DSM_COG_10_N23_00_W65_00_DEM/Copernicus_DSM_COG_10_N00_00_E006_00_DEM.tif'

In [246]:
requests.get(f'https://{dem_bucket}.s3.amazonaws.com/{key_list[0]}')

True

In [235]:
import os
# os.chdir('../Data/Training/Copernicus')

In [247]:
bad_keys = []
for k in key_list:
    ind = key_list.index(k)
    if ind % 25 == 0:
        print(str(ind) + ' done.')
        print(str(len(key_list) - ind) + ' left.')
    
    resp = requests.get(f'https://{dem_bucket}.s3.amazonaws.com/{k}')
    
    if resp.status_code != 404:
        
        nc4_ds = netCDF4.Dataset(k, memory = resp.content)
        store = xr.backends.NetCDF4DataStore(nc4_ds)
        DS = xr.open_dataset(store)

        DS.to_netcdf(k + '.nc')
    
    elif resp.status_code == 404:
        bad_keys.append(k)

0 done.
1620 left.
25 done.
1595 left.
50 done.
1570 left.
75 done.
1545 left.
100 done.
1520 left.
125 done.
1495 left.
150 done.
1470 left.
175 done.
1445 left.
200 done.
1420 left.
225 done.
1395 left.
250 done.
1370 left.
275 done.
1345 left.


KeyboardInterrupt: 

In [229]:
!ls

Convert_and_Upload_VIIRS.ipynb
Fire_History
LST
Play-with_VIIRS.ipynb
Pull_Ancillary.ipynb
Pull_VIIRS_DATA-Copy1.ipynb
Pull_VIIRS_DATA_OLD.ipynb
Sample_Training_Data.ipynb
SatelliteVu-AWS-Disaster-Response-Hackathon
composite_goes.ipynb
fire_shp.ipynb
fires_saved.nc.aux.xml
goes2go
laads-data-download.py
process_GOES_github.ipynb.txt
process_GOES_github.rtf.sb-db243f12-PWMyEM
pull_GOES.ipynb
pull_training_data.ipynb
random_LST.nc
src
temp


In [181]:
def get_s3_keys(bucket, s3_client, prefix = ''):
    
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

        
    
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [182]:
dem_keys = [key for key in get_s3_keys(dem_bucket, s3_client)]

KeyboardInterrupt: 

In [183]:
len(dem_keys)

1000

In [173]:
dem_contents

[{'Key': '2019/Copernicus_DSM_COG_10_N00_00_E006_00_DEM/Copernicus_DSM_COG_10_N00_00_E006_00_DEM.tif',
  'LastModified': datetime.datetime(2022, 4, 4, 15, 44, 52, tzinfo=tzutc()),
  'ETag': '"e0c8b05f7999eedd83ba85e5c94cb670"',
  'Size': 3706379,
  'StorageClass': 'STANDARD'},
 {'Key': '2019/Copernicus_DSM_COG_10_N00_00_E009_00_DEM/Copernicus_DSM_COG_10_N00_00_E009_00_DEM.tif',
  'LastModified': datetime.datetime(2022, 4, 4, 15, 44, 52, tzinfo=tzutc()),
  'ETag': '"9797ee1a2cb089cbfa5c753cdfbfc0bb"',
  'Size': 27322465,
  'StorageClass': 'STANDARD'},
 {'Key': '2019/Copernicus_DSM_COG_10_N00_00_E010_00_DEM/Copernicus_DSM_COG_10_N00_00_E010_00_DEM.tif',
  'LastModified': datetime.datetime(2022, 4, 4, 15, 44, 52, tzinfo=tzutc()),
  'ETag': '"52d2cfa58be0b3bca74eecb11225557e"',
  'Size': 48574244,
  'StorageClass': 'STANDARD'},
 {'Key': '2019/Copernicus_DSM_COG_10_N00_00_E011_00_DEM/Copernicus_DSM_COG_10_N00_00_E011_00_DEM.tif',
  'LastModified': datetime.datetime(2022, 4, 4, 15, 44, 52, t

# MODIS DATA - > Huge pain

In [136]:
import pyorc
import pandas as pd

with open(r"temp/03e78e57-63cf-4a66-a5bd-11371f7f6dd9.orc", "rb") as orc_file:
    reader = pyorc.Reader(orc_file)
    orc_data = reader.read()
    orc_schema = reader.schema

columns = list(orc_schema.fields)
df = pd.DataFrame(data=orc_data, columns=columns)

In [140]:
df

,bucket,key,size,last_modified_date,storage_class
0,astraea-opendata,MCD43A4.006/08/05/2014279/MCD43A4.A2014279.h08...,7526998,2019-11-04 11:26:01+00:00,INTELLIGENT_TIERING
1,astraea-opendata,MCD43A4.006/08/05/2014279/MCD43A4.A2014279.h08...,99036,2019-11-04 11:25:41+00:00,STANDARD
2,astraea-opendata,MCD43A4.006/08/05/2014280/MCD43A4.A2014280.h08...,7237290,2019-11-04 11:32:56+00:00,INTELLIGENT_TIERING
3,astraea-opendata,MCD43A4.006/08/05/2014280/MCD43A4.A2014280.h08...,90168,2019-11-04 11:32:50+00:00,STANDARD
4,astraea-opendata,MCD43A4.006/08/05/2014280/MCD43A4.A2014280.h08...,7279383,2019-11-04 11:32:59+00:00,INTELLIGENT_TIERING
...,...,...,...,...,...
399995,astraea-opendata,MCD43A4.006/08/09/2014044/MCD43A4.A2014044.h08...,24159,2019-11-04 11:02:18+00:00,STANDARD
399996,astraea-opendata,MCD43A4.006/08/09/2014044/MCD43A4.A2014044.h08...,92446,2019-11-04 11:02:22+00:00,STANDARD
399997,astraea-opendata,MCD43A4.006/08/09/2014044/MCD43A4.A2014044.h08...,25247,2019-11-04 11:02:18+00:00,STANDARD
399998,astraea-opendata,MCD43A4.006/08/09/2014044/MCD43A4.A2014044.h08...,81788,2019-11-04 11:02:23+00:00,STANDARD


In [139]:
df['key'][0]

'MCD43A4.006/08/05/2014279/MCD43A4.A2014279.h08v05.006.2016154053557_B07.TIF'

In [142]:
'astraea-opendata-inventory/astraea-opendata/modis-assets-MCD43A4/MCD43A4.006/08/05/2014279/MCD43A4.A2014279.h08v05.006.2016154053557_B07.TIF'

'astraea-opendata-inventory/astraea-opendata/modis-assets-MCD43A4/MCD43A4.006/08/05/2014279/MCD43A4.A2014279.h08v05.006.2016154053557_B07.TIF'

# WorldCover Scratch

In [5]:
cover_bucket = 'esa-worldcover'

cover_contents = s3_client.list_objects(Bucket=cover_bucket)['Contents']

cover_readme = cover_contents[0]['Key']

First, examine the grid that ESA threw down. I want to find out how many images are in each grid cell for this dataset.

In [60]:
grid = gpd.read_file('temp/esa_worldcover_2020_grid.geojson')
# Bounding box of CONUS
xmin, ymin, xmax, ymax = -124.848974,24.396308,-66.885444,49.384358
conus_grid = grid.cx[xmin:xmax, ymin:ymax]
conus_grid.to_file('temp/worldcover_2020_grid_conus.geojson')



In [6]:
cover_contents

[{'Key': 'readme.html',
  'LastModified': datetime.datetime(2021, 10, 25, 9, 48, 50, tzinfo=tzutc()),
  'ETag': '"8b7361f4c8bcfa9a6906f3bc251a11d4"',
  'Size': 2830,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': '91d380b3cead28df927c824731b0173701336cd8d67b0679d7166288f3850f38'}},
 {'Key': 'v100/2020/ESA_WorldCover_10m_2020_v100_Map_AWS.vrt',
  'LastModified': datetime.datetime(2021, 10, 18, 9, 40, 44, tzinfo=tzutc()),
  'ETag': '"1006bf44330fc1e20b6ee1999fbc5a61"',
  'Size': 1324634,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': '91d380b3cead28df927c824731b0173701336cd8d67b0679d7166288f3850f38'}},
 {'Key': 'v100/2020/docs/ESAWorldCover_ColorLegend.lyr',
  'LastModified': datetime.datetime(2021, 10, 19, 22, 27, 8, tzinfo=tzutc()),
  'ETag': '"a77cb56931f265d587eeb3e9b36457fd"',
  'Size': 8192,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': '91d380b3cead28df927c824731b0173701336cd8d67b0679d7166288f3850f38'}},
 {'Key': 'v100/2020/docs/ESAWorldCover_ColorLegend.qml',
  'LastModified'

In [63]:
cover_contents[10]['Key'].split('/')[-1].split('_')

['ESA', 'WorldCover', '10m', '2020', 'v100', 'N00E006', 'Map.tif']

In [91]:
def extract_wc_info(obj_list):
    return_list = []
    
    for obj in obj_list:
        key = obj['Key']
        info = key.split('/')[-1].split('_')

        # Catch any non tif items
        if not info[-1].endswith('.tif'):
            continue

        cell = info[-2]
        return_list.append(cell)
        
    return return_list

In [92]:
cells = np.array(extract_wc_info(cover_contents))



In [99]:
conus_grid

,ll_tile,geometry
1640,N24W075,"POLYGON ((-75.00000 24.21125, -74.99055 24.136..."
1641,N24W078,"POLYGON ((-78.00000 26.06029, -77.93569 26.084..."
1648,N24W081,"MULTIPOLYGON (((-81.00000 27.00000, -79.77661 ..."
1649,N24W084,"POLYGON ((-81.00000 27.00000, -81.00000 24.399..."
1650,N27W090,"POLYGON ((-90.00000 30.00000, -87.40934 30.000..."
...,...,...
2289,N24W102,"POLYGON ((-102.00000 27.00000, -99.00000 27.00..."
2290,N24W099,"POLYGON ((-99.00000 27.00000, -97.07819 27.000..."
2296,N24W105,"POLYGON ((-105.00000 27.00000, -102.00000 27.0..."
2297,N24W108,"POLYGON ((-108.00000 27.00000, -105.00000 27.0..."


In [98]:
cells

['N00E006',
 'N00E009',
 'N00E012',
 'N00E015',
 'N00E018',
 'N00E021',
 'N00E024',
 'N00E027',
 'N00E030',
 'N00E033',
 'N00E036',
 'N00E039',
 'N00E042',
 'N00E045',
 'N00E072',
 'N00E093',
 'N00E096',
 'N00E099',
 'N00E102',
 'N00E105',
 'N00E108',
 'N00E111',
 'N00E114',
 'N00E117',
 'N00E120',
 'N00E123',
 'N00E126',
 'N00E129',
 'N00E132',
 'N00E171',
 'N00W051',
 'N00W054',
 'N00W057',
 'N00W060',
 'N00W063',
 'N00W066',
 'N00W069',
 'N00W072',
 'N00W075',
 'N00W078',
 'N00W081',
 'N00W090',
 'N00W093',
 'N00W159',
 'N03E000',
 'N03E003',
 'N03E006',
 'N03E009',
 'N03E012',
 'N03E015',
 'N03E018',
 'N03E021',
 'N03E024',
 'N03E027',
 'N03E030',
 'N03E033',
 'N03E036',
 'N03E039',
 'N03E042',
 'N03E045',
 'N03E048',
 'N03E072',
 'N03E078',
 'N03E093',
 'N03E096',
 'N03E099',
 'N03E102',
 'N03E105',
 'N03E108',
 'N03E111',
 'N03E114',
 'N03E117',
 'N03E120',
 'N03E123',
 'N03E126',
 'N03E156',
 'N03E162',
 'N03E168',
 'N03E171',
 'N03W003',
 'N03W006',
 'N03W009',
 'N03W012',
 'N0

In [100]:
conus_cells = list(conus_grid['ll_tile'])

In [113]:
conus_cells_key = [c for c in cells if c in conus_cells]

In [115]:
len(conus_cells)

153

In [119]:
checks = [c for c in conus_cells if c not in conus_cells_key]

In [127]:
conus_grid.set_index('ll_tile').loc[checks].to_file('temp/check_grid.geojson')